# Test

In [ ]:
import os
import dotenv
from scholarly import scholarly
from langchain_community.tools.google_scholar import GoogleScholarQueryRun
from langchain_community.utilities.google_scholar import GoogleScholarAPIWrapper

### Basic

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langserve import add_routes

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Create model
# model = ChatOpenAI()

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | llm | parser


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

### Benchmark

In [ ]:
# Vector db #1
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.document_loaders import UnstructuredHTMLLoader, BSHTMLLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from langchain.embeddings import OllamaEmbeddings  

# DATA_PATH="/mnt/c/Users/beene/Downloads/papers/"
DATA_PATH="/mnt/c/Users/beene/Downloads/tests/"
DB_PATH = "./vectorstores/db/"

#load the LLM
def load_llm():
    llm = Ollama(
        model="llama3",
        base_url="http://localhost:11434",
        verbose=True,
        callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    )
    return llm

def create_vector_db():
    loader = PyPDFDirectoryLoader(DATA_PATH)
    documents = loader.load()
    print(f"Processed {len(documents)} pdf files")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    texts=text_splitter.split_documents(documents)
    vectorstore = Chroma.from_documents(documents=texts, embedding=OllamaEmbeddings(), persist_directory=DB_PATH)

def load_vector_db():
    return Chroma(persist_directory=DB_PATH, embedding_function=OllamaEmbeddings())

if __name__=="__main__":
    llm = load_llm()
    # create_vector_db()
    vectorstore = load_vector_db()

In [ ]:
from fastapi import FastAPI

from langserve import add_routes

app = FastAPI(
    title="LangChain Server",
    version="1.0",
    description="Spin up a simple api server using Langchain's Runnable interfaces",
)

add_routes(
    app,
    llm,
    path="/llm",
)
# add_routes(
#     app,
#     ChatAnthropic(model="claude-3-haiku-20240307"),
#     path="/anthropic",
# )

async def main():
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

if __name__ == "__main__":
    await main()

In [ ]:
# Vector db #2
import chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings  

persistent_client = chromadb.PersistentClient()
collection = persistent_client.create_collection("default")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

# langchain_chroma = Chroma(
#     client=persistent_client,
#     collection_name="default",
#     embedding_function=OllamaEmbeddings(),
# )

In [ ]:
langchain_chroma.get()

In [ ]:
test = chromadb.PersistentClient(
    path = "./vectorstores/db/"
)

In [ ]:
test2 = Chroma(
    client = test,
    collection_name = "default",
    embedding_function = GPT4AllEmbeddings()
)

In [ ]:
test2.get()

In [ ]:
# Contextual compression retriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

compressor = FlashrankRerank(top_n = 1)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "What did the president say about Ketanji Jackson Brown"
)
compressed_docs

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multiquery_retriever = MultiQueryRetriever.from_llm( 
    retriever=retriever,
    llm=llm,
)
compression_retriever.invoke("What did the president say about Ketanji Jackson Brown")

In [ ]:
#import required dependencies
from langchain import hub
from langchain.embeddings import GPT4AllEmbeddings
from langchain.vectorstores import Chroma
import chainlit as cl
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
# Set up RetrievelQA model
QA_CHAIN_PROMPT = hub.pull("rlm/rag-prompt-mistral")


def retrieval_qa_chain(llm,vectorstore):
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
        return_source_documents=True,
    )
    return qa_chain


def qa_bot(): 
    llm=load_llm() 
    vectorstore = load_vector_db()

    qa = retrieval_qa_chain(llm,vectorstore)
    return qa 

@cl.on_chat_start
async def start():
    chain=qa_bot()
    msg=cl.Message(content="Firing up the research info bot...")
    await msg.send()
    msg.content= "Hi, welcome to research info bot. What is your query?"
    await msg.update()
    cl.user_session.set("chain",chain)

@cl.on_message
async def main(message):
    chain=cl.user_session.get("chain")
    cb = cl.AsyncLangchainCallbackHandler(
    stream_final_answer=True,
    answer_prefix_tokens=["FINAL", "ANSWER"]
    )
    cb.answer_reached=True
    # res=await chain.acall(message, callbacks=[cb])
    res=await chain.acall(message.content, callbacks=[cb])
    print(f"response: {res}")
    answer=res["result"]
    answer=answer.replace(".",".\n")
    sources=res["source_documents"]

    if sources:
        answer+=f"\nSources: "+str(str(sources))
    else:
        answer+=f"\nNo Sources found"

    await cl.Message(content=answer).send() 

# Main

In [171]:
#!/usr/bin/env python
# to ignore the deprecation warnings and info
import logging
import warnings
from langchain._api import LangChainDeprecationWarning
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)
warnings.filterwarnings('ignore')
logging.disable(logging.INFO)

# import the necessary packages
import uuid
import sys, os
import chromadb
import numpy as np
from pathlib import Path
from langchain import hub
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.embeddings import GPT4AllEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.chains import RetrievalQA
from nemoguardrails import RailsConfig, LLMRails

# PATHs
DATA_PATH="/mnt/c/Users/beene/Downloads/papers/"
DB_PATH = "./chroma"

# Load or make the client
print("Loading client ... \n")
client = chromadb.PersistentClient(path = DB_PATH)

# Preload the LLM
llm = Ollama(
    model = "llama3",
    base_url = "http://localhost:11434",
    temperature = 0,
    verbose = True,
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]),
)

def load_guardrails():
    # Nemo Guardrails
    config = RailsConfig.from_path("./config/config.yml")
    rails = LLMRails(config, llm = llm)
    return rails

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout    

# Create a ChatPromptTemplate 
def create_prompt_template():
    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            "You are a research assistant specializing in question-answering tasks. "
            "Use the provided context to answer the question concisely. "
            "If you don't know the answer, simply state that you don't know. "
            "Keep your response to a maximum of three sentences. "
            "Keep following 3 instructions in the structured format. "
            "1. Summarize the main points in bullet points, using •. This is the highest priority. "
            "2. Provide a detailed description with sufficient detail for understanding, without unnecessary elaboration. "
            "3. Include any additional relevant information, if available, but keep it brief and optional. "
        ),
        (
            "user", 
            "Question: {question}\n" 
            "Context: {context}\n"
            "Answer: ",
        ),
        (
            "assistant", 
            "**Main Points:**\n"
            "**Detailed Description:**\n"
            "**Additional Information:**\n"
        )
    ])
    return prompt

def count_pdfs():
    import os
    count = 0
    for root, dirs, files in os.walk(DATA_PATH):
        count += len([fn for fn in files if fn.endswith(".pdf")])
    return count

# Vector DB functions
def create_documents():
    '''Upload the PDFs in the DATA_PATH directory'''
    loader = PyPDFDirectoryLoader(DATA_PATH, silent_errors = False)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50)
    texts = text_splitter.split_documents(documents)
    return texts

def create_embeddings(documents):
    '''Create embeddings for the documents in the DATA_PATH directory'''
    embedder = GPT4AllEmbeddings()
    return [embedder.embed_query(doc.page_content) for doc in documents]

def clear_db():
    list_collections = client.list_collections()
    if len(list_collections):
        for i in list_collections:
            client.delete_collection(i.dict()["name"])

def create_vector_db():
    '''Create a vector db from the documents in the DATA_PATH directory'''
    # Make original paper db and duplicated db for search purposes
    # , which are saved to the DB_PATH directory
    _ = client.create_collection(
        name = "Original",
        metadata={"hnsw:space": "cosine"},
    )
    _ = client.create_collection(
        name = "Search",
    )

def add_documents_to_db():
    # Load the vector db client
    collection = client.get_collection("Original")

    # Create documents and embeddings
    documents = create_documents()
    embeddings = create_embeddings(documents)

    # Add the documents and embeddings to the collection
    collection.add(
        ids = [str(uuid.uuid4()) for _ in documents],
        documents = [doc.page_content for doc in documents],
        metadatas = [doc.metadata for doc in documents],
        embeddings = embeddings
    )

def implementation_db():
    '''Implementation of the above functions'''
    # Load LM    
    print(
        "\nCreate vector database ..."
        "It could take a while depending on your dataset."
    )
    with HiddenPrints():
        clear_db() # Clean up the existing db
        create_vector_db()
        add_documents_to_db()
    print("Done.\n\n")

def load_vector_db(collection_name):
    '''Load the vector db for chaining purposes'''
    return Chroma(
        client = client,
        persist_directory = DB_PATH,
        collection_name = collection_name,
        embedding_function = GPT4AllEmbeddings(),
        collection_metadata={"hnsw:space": "cosine"},
    )

def extract_source_document(response):
    '''Extract the source document from the response'''
    source_document = response.get("source_documents")[0].dict()
    return source_document["metadata"]["source"]

def delete_searched_document(response):
    '''Delete the searched document from the db'''
    collection = client.get_collection("Search")
    collections = collection.get() # get a dict db

    # find the id
    source_document = extract_source_document(response)
    ids = collections["ids"]
    ids_to_delete = []
    for i in range(len(ids)):
        if collections["metadatas"][i]["source"] == source_document:
            ids_to_delete.append(ids[i])

    collection.delete(ids = ids_to_delete)


# Retrieval QA functions
# Contextual compression retriever
def get_retrieval():
    compressor = FlashrankRerank(top_n = 1)
    chroma_db = load_vector_db("Search")
    return ContextualCompressionRetriever(
        base_compressor = compressor, 
        base_retriever = chroma_db.as_retriever()
    )

def get_chain(rail_llm):
    guardrails = load_guardrails()
    prompt = create_prompt_template()
    retriever = get_retrieval()
    return RetrievalQA.from_chain_type(
        rail_llm,
        retriever = retriever,
        chain_type_kwargs = {"prompt": prompt},
        return_source_documents = True,
    )

def check_relevance_score(retriever, query):
    '''Check the relevance score of the query'''
    threshold = 0.5
    retrieved_documents = retriever.invoke(query)
    if len(retrieved_documents):
        return retrieved_documents[0].metadata["relevance_score"] >= threshold
    else:
        return False

# Make the response pretty
def remove_empty_lines(result):
    copy_result = result.copy()
    for i in result:
        if len(i) == 0:
            copy_result.remove(i)
    indices = []
    for i in range(len(copy_result)):
        if copy_result[i].endswith("**") and i != 0:
            indices.append(i)
    for i in indices[::-1]:
        copy_result.insert(i, "")         
    return copy_result

def print_bunch():
    return print(
        "---------------------------------------"
        "---------------------------------------"
        "---------------------------------------"
    )

def pprint_response(responses, response):
    result = remove_empty_lines(response["result"].split("\n")[1:])
    file_path = response["source_documents"][0].metadata["source"]
    file_name = Path(file_path).name
    print_bunch()
    print(f"\nDocuments #{len(responses)}")
    print(f"The source document: {file_name}\n\n")
    _ = [print(phrase) for phrase in result]
    print_bunch()

# Main functions
def ask_user_to_create_db():
    while True:
        user_input = input("Do you want to create a new database? (y/n): ")
        if user_input == "n":
            break
        elif user_input == "y":
            implementation_db()
            break
        print("Invalid input. Please enter \'y\' or \'n\'.")

def ask_how_many_documents():
    # print(f"\n***Total documents: {count_pdfs()}***")
    while True:
        iter_num = input(
            f"How many documents do you want to search? (<= {count_pdfs()}): " 
        )
        if iter_num.isdigit() and int(iter_num) <= count_pdfs():
            return int(iter_num)
        print("Invalid input. Please enter a number.")

def prepare_db():
    '''Duplicate the existing vector db for iterative search purposes'''
    print("\nPreparing a database for a search purpose ... ")
    # Clear the search db
    client.delete_collection("Search")

    # Load the vector db client
    collection = client.get_collection("Original")
    collection2 = client.create_collection(
        name = "Search", 
        metadata={"hnsw:space": "cosine"},
    )
    
    # Dupliacte a collection
    total_documents_count = collection.count()
    batch_size = 10
    for i in range(0, total_documents_count, batch_size):
        batch = collection.get(
            include=["metadatas", "documents", "embeddings"],
            limit=batch_size,
            offset=i
        )
        collection2.add(
            ids=batch["ids"],
            documents=batch["documents"],
            metadatas=batch["metadatas"],
            embeddings=batch["embeddings"]
        )
    print("Done.\n")

def response_loop(iter_num):
    assert iter_num <= count_pdfs()
    '''Iterative search for documents'''
    # params
    rails = load_guardrails()
    retriever = get_retrieval()
    chain = get_chain(rails.llm)
    
    # Implement the search loop
    responses = []
    query = input("\nWhat is your question?: ")
    for _ in range(iter_num):
        # Check if the search db is empty
        if client.get_collection("Search").count() == 0:
            print("\n=======================================")
            print("No documents left to search.")
            print("=======================================\n")
            break

        # Check the relevance score of the query
        if check_relevance_score(retriever, query):
            # Get response
            with HiddenPrints():
                response = chain.invoke(query)
            
            # Print the response (Custom)
            responses.append(response)
            pprint_response(responses, response)

            # Delete the returned document from the db
            delete_searched_document(response)
        else: 
            # Quit the search loop if the relevance score is below the threshold
            print("\n=======================================")
            print("A searched document is not relevant.")
            print("=======================================\n")
            break
    print("\n==================================")    
    print(f"Total number of search is {len(responses)}.")
    print("Searching is done.")
    print("==================================\n")
    return responses

def ask_if_more_to_ask():
    while True:
        user_input = input("Do you want to ask more questions? (y/n): ")
        if user_input == "n":
            print("\n\nGoodbye!")
            quit()
        elif user_input == "y":
            return
        print("Invalid input. Please enter \'y\' or \'n\'.")

Loading client ... 



In [174]:
    rails = load_guardrails()
    retriever = get_retrieval()
    chain = get_chain(rails.llm)

In [177]:
test = chain.invoke("What is EEG characteristics of conflict cognition?")

Here is the answer:

**Main Points:**

• EEG characteristics of conflict cognition are associated with increased alpha and theta power, and decreased beta power
• Alpha and theta oscillations may reflect cognitive conflict and emotional arousal, while beta oscillations may indicate executive control and attentional resources

**Detailed Description:**
According to [24] Knyazev's research on motivation, emotion, and inhibitory control, EEG characteristics of conflict cognition are characterized by increased alpha (8-12 Hz) and theta (4-8 Hz) power, and decreased beta (13-30 Hz) power. Alpha and theta oscillations may reflect cognitive conflict and emotional arousal, while beta oscillations may indicate executive control and attentional resources.

**Additional Information:**
For a more comprehensive understanding of brain rhythms, I recommend [25] Buzsaki's book "Rhythms of the Brain", which provides an in-depth overview of neural oscillations and their roles in various cognitive proces

In [178]:
test

{'query': 'What is EEG characteristics of conflict cognition?',
 'result': 'Here is the answer:\n\n**Main Points:**\n\n• EEG characteristics of conflict cognition are associated with increased alpha and theta power, and decreased beta power\n• Alpha and theta oscillations may reflect cognitive conflict and emotional arousal, while beta oscillations may indicate executive control and attentional resources\n\n**Detailed Description:**\nAccording to [24] Knyazev\'s research on motivation, emotion, and inhibitory control, EEG characteristics of conflict cognition are characterized by increased alpha (8-12 Hz) and theta (4-8 Hz) power, and decreased beta (13-30 Hz) power. Alpha and theta oscillations may reflect cognitive conflict and emotional arousal, while beta oscillations may indicate executive control and attentional resources.\n\n**Additional Information:**\nFor a more comprehensive understanding of brain rhythms, I recommend [25] Buzsaki\'s book "Rhythms of the Brain", which provide

### Nemo Guardrails

In [ ]:
from nemoguardrails import RailsConfig
from nemoguardrails.integrations.langchain.runnable_rails import RunnableRails
# Nemo Guardrails
# ... initialize `some_chain`
config = RailsConfig.from_path("./config/config.yml")

# Using LCEL, you first create a RunnableRails instance, and "apply" it using the "|" operator
guardrails = RunnableRails(config)
# chain_with_guardrails = guardrails | some_chain


# RUN

In [ ]:
# db implementation
if __name__=="__main__":
    llm = load_llm()
    client = chromadb.PersistentClient(path = DB_PATH)
    # implementation_db(client)

In [ ]:
# Make db for search and create chain
duplicate_db(client)
retriever = get_retrieval()
chain = retrieval_qa_chain()

In [ ]:
iter_num = np.round(count_pdfs()*0.1).astype(int)
query = "What kind of experiments are implemented where PLV method is used?"
responses = response_loop(1)

In [125]:
test = chain.invoke("How can PLV be calculated?")

Here's my response:

**Main Points:**

* PLV (Phase-Locked Value) is calculated using instantaneous phase angles obtained by applying the Hilbert transformation to bandpass-filtered data.
* The formula for calculating PLV between two signals A and B is: PLV = 1/T ∑t=1 e^(-i(φA(t) - φB(t)))
* Three temporal features are extracted from each PLV time series: mean, variability (standard deviation), and sample entropy.

**Detailed Description:**

To calculate the PLV, instantaneous phase angles are obtained by applying the Hilbert transformation to the bandpass-filtered data. The formula for calculating PLV between two signals A and B is then used, where ϕA(t) and ϕB(t) are the instantaneous phase angles of each EEG signal.

**Additional Information:**

Sample entropy is a non-linear measure that quantifies the degree of complexity in a time series (Richman and Moorman, 2000).